In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!java -version
!pip install findspark

In [26]:
import os 
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row
import pyspark.sql.functions as f

## You can add more config while building 
spark = SparkSession.builder.master("local[8]").\
                    config("spark.app.name","session_one").\
                    getOrCreate() #number of threads = 16

querycounter = 0 #global query counter

In [ ]:
df = spark.read.csv("testpeople.csv",header=True,inferSchema=True)
df.show()

In [ ]:
def listOfFrequencyTables(df): #take main dataframe, generate frequency dataframes
  histograms = []
  for col in df.dtypes:
      h=df.groupBy(col[0]).count()
      h = h.sort(desc("count"))
      histograms.append(h)
      h.show() #comment this line to suppress output
  return histograms
histograms = listOfFrequencyTables(df)

In [ ]:
def getDecompFromTopFrequencies(histograms):
  clusterlst=[]
  for i in range(len(histograms)): #query database with top values of all columns
    d= str(histograms[i].first()) #value of the first row
    print(d)
    d = d.split(",")[0].split('=')[1] #the splits are for formatting the string
    print(d)
    #print("d before:",d)
    if "'"  in d:
      d = d.split("'")[1]

    #print("d after:",d)
    #print(type(d))
    cname = str(histograms[i][0]).split("'")[1]
    print(cname,"=",d)
    
    data = (df.filter(col(cname) == d))
    
    data.show(15)
    clusterlst.append(data)
  return clusterlst

clusterlst = getDecompFromTopFrequencies(histograms)


In [64]:
def are_dfs_equal(df1, df2): #this works, i tested it
  res = df1.subtract(df2) #set subtraction on the two dataframes. 
  if res.count() == 0: #subtraction yielded empty set
    print("dataframes are equal")
    return True
  else:
    print("error! these rows are not in the union of your queries:")
    res.show() #show which tuples are not included in your query union
    return False

In [63]:
from functools import reduce
from pyspark.sql.functions import lit
from pyspark.sql import DataFrame
def getDecompUsingFreqTable(df,freqdf): #takes original database and one frequency table as input, returns union of all queried dataframes as output
  print("this is frequency table:")
  freqdf.show()
  cname = freqdf.columns[0]
  valuelist = (freqdf.select(freqdf.columns[0]).rdd.flatMap(lambda x: x).collect()) #list of all values of frequency column

  unionlst = []
  for v in valuelist: #each unique value in the freq. table is used as a query
    result = df.filter(col(cname) == v)
    colnames = df.columns
    querystr = cname + "=" + str(v)
    containsquery = False
    for c in colnames:
      print("colname is:",c)
      if "query" in c:
        print("query col found")
        containsquery= True
    
    if(containsquery): #check if query column already exists in the input
      newquery = str(result.select(col("query")).first()).split("'")[1] + "," + querystr
      print('newres:',newquery)
      newres=result.withColumn("query",lit(newquery))
    else:
      print("creating query column:")
      newres=result.withColumn("query",lit(querystr))
    print(querystr)
    #newres.show()
    unionlst.append(newres)
  unn = reduce(DataFrame.unionAll, unionlst) #put all queried dataframes back together as one
  print("union:")
  #unn.show()

  return unn


union = (getDecompUsingFreqTable(df,histograms[0])) #function call with 'Name' frequency table
print("equality result:")
subdf = are_dfs_equal(df,union.drop('query')) #checks if union of queries covers whole database
print(subdf)
print("\nnext run:")
union1 = (getDecompUsingFreqTable(union,histograms[1]))
union2 = (getDecompUsingFreqTable(union1,histograms[2]))

print("after queries:\n")
union.show(10,False)
union1.show(10,False)
union2.show(10,False)


this is frequency table:
+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

colname is: Name
colname is: city
colname is: age
creating query column:
Name=Nick
colname is: Name
colname is: city
colname is: age
creating query column:
Name=Mary
colname is: Name
colname is: city
colname is: age
creating query column:
Name=John
union:
equality result:
dataframes are equal
True

next run:
this is frequency table:
+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    4|
|rotterdam|    4|
|amsterdam|    2|
+---------+-----+

colname is: Name
colname is: city
colname is: age
colname is: query
query col found
newres: Name=Nick,city=rotterdam
city=rotterdam
colname is: Name
colname is: city
colname is: age
colname is: query
query col found
newres: Name=Nick,city=utrecht
city=utrecht
colname is: Name
colname is: city
colname is: age
colname is: query
query col found
newres: Name=Nick,city=amsterdam
city=amsterdam
union:
this is frequency tabl

In [9]:
def are_dfs_equal(df1, df2): #this works, i tested it
  res = df1.subtract(df2) #set subtraction on the two dataframes. 
  if res.count() == 0: #subtraction yielded empty set
    print("dataframes are equal")
    return True
  else:
    print("error! these rows are not in the union of your queries:")
    res.show() #show which tuples are not included in your query union
    return False

In [ ]:
listOfFrequencyTables(clusterlst[0])

+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
+----+-----+

+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    2|
|amsterdam|    1|
|rotterdam|    1|
+---------+-----+

+---+-----+
|age|count|
+---+-----+
| 20|    2|
| 22|    1|
| 21|    1|
+---+-----+



[DataFrame[Name: string, count: bigint],
 DataFrame[city: string, count: bigint],
 DataFrame[age: int, count: bigint]]

In [ ]:
#ignore this stuff lol

'''
  clusterlst=[]
  for i in range(len(df)): #query database with top values of all columns
    d= str(histograms[i].first()) #value of the first row
    print(d)
    d = d.split(",")[0].split('=')[1] #the splits are for formatting the string
    print(d)
    #print("d before:",d)
    if "'"  in d:
      d = d.split("'")[1]

    #print("d after:",d)
    #print(type(d))
    cname = str(histograms[i][0]).split("'")[1]
    print(cname,"=",d)
    
    data = (df.filter(col(cname) == d))
    
    data.show(15)
    clusterlst.append(data)
  return clusterlst

clusterlst = getDecompFromTopFrequencies(histograms)
'''
'''
for i in range(len(histograms)):
  d= str(histograms[i].first()) #first row
  d = d.split(",")[0].split('=')[1]
  if "'"  in d:
    d = d.split("'")[1]
  cname = str(histograms[i][0]).split("'")[1] #column name
  data = filterdf(df,cname,d)
  data.show()
'''

In [58]:
'''
from pyspark.sql import Row
import pyspark.sql.functions as f
dat = [Row('Name=Mary'),Row('Name=John')]
cln = ['query']
dummy = spark.createDataFrame(data=dat, schema = cln)
strq = str(dummy.select(col("query")).first()).split("'")[1]

print(strq)
'''


Name=Mary
